In [1]:
from AlgoTools import AlgoTools
from Trader import Trader
import pandas as pd
import time
import datetime
import pickle
import numpy as np

# Processing the data
- The data has already been grouped into days and converted to date time objects
- Objective is to shuffle the days, then create a train-test split as I will be training on indivuals days

In [125]:
with open('SP-groupedByDays.pkl', 'rb') as f:
    spDaily = pickle.load(f)

In [126]:
with open('spydf.pickle', 'rb') as f:
    tickData = pickle.load(f)

In [127]:
import random
random.shuffle(spDaily)

In [128]:
trainLen = int(len(spDaily)*0.75)

In [129]:
trainData=spDaily[:trainLen]
testData=spDaily[trainLen:]

In [130]:
tempData=spDaily[:10]

# Creating the Algorithim

In [131]:
algoTools = AlgoTools(tickData)

In [132]:
def Algo(params):
    data = trainData
    r, a_L, a_S, E_e, stopProf, stopLoss = params
    trader = Trader()
    #for each day in the data
    for d in range(len(data)):
        #for each time stamp in the day
        index = 1
        
        for t in range(data[d].index[2] , data[d].index[len(data[d])-1]):
            index = index + 1
            
            #close out because market is closing                
            ti = tickData.loc[t,"Time2"].hour*100 + tickData.loc[t,"Time2"].minute
            if (1513<ti<1520 or 2330<ti<2359):
                if trader.getLastTradePos() != 0:
                    trader.closeOut(data[d].iloc[index,:])
                continue
            
            


            
            #if the acceleration is greater than or equal to acceleration constant Long
            if (algoTools.getAccel(t) >= a_L or algoTools.getRate(t) >= r) and trader.getLastTradePos() == 0:
                trader.goLong(data[d].iloc[index,:])
                continue
            
            #same as previous statment but for shorting
            if  ( algoTools.getAccel(t) <= a_S or algoTools.getRate(t) <= -r ) and trader.getLastTradePos() == 0:
                trader.goShort(data[d].iloc[index,:])
                continue
            
            
            
            
           
            
            #-----Closing Conditions------
            
            
            #as per calclus reasoing if the first deravitive is = 0 it has found a local extrema. therfore close out 
            #E_e is error of extrema
            if algoTools.getRate(t) <= E_e and trader.getLastTradePos() != 0:
                trader.closeOut(data[d].iloc[index,:])
                continue
               
            
            #stop profit (in points)
            if (trader.getLastTradePos() == 1 and data[d].loc[t,"price"] - trader.getLastTradePrice() >= stopProf ) or (trader.getLastTradePos() == -1 and trader.getLastTradePrice() - data[d].loc[t,"price"] >= stopProf):
                trader.closeOut(data[d].iloc[index,:])
                continue
            
            #stop loss (in points)
            if (trader.getLastTradePos() == 1 and data[d].loc[t,"price"] - trader.getLastTradePrice() <= stopLoss) or (trader.getLastTradePos() == -1 and trader.getLastTradePrice() - data[d].loc[t,"price"] <= stopLoss ):
                trader.closeOut(data[d].iloc[index,:])
                continue
                
            
            
                
            
    return trader

In [133]:
trader = Algo([1,0.5,-0.5,0.01,2,0.5])

In [134]:
trader.getTradeInfo()

{'Total Return': 225529.99999999529,
 'Max Return': 364.9999999999977,
 'Min Return': -18055.000000000007}

In [135]:
trader.getLR()

0    0.396819
dtype: float64

In [ ]:
from scipy.optimize import minimize



# Initial guess for the parameters
initial_guess = [1,0.5,-0.5,0.01,2,0.5]

# Perform the minimization
result = minimize(Algo, initial_guess)

# Output the result
print("Optimized parameters:", result.x)
print("Function value at optimized parameters:", result.fun)